In [18]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import lxml
import gpxpy, gpxpy.gpx

### Introduction
The plan here is to use libraries to parse the .gpx files and aggregate their contents. Afterwards, manipulating the data into the .csv and running some basic analysis. 

In [19]:
wd = r'C:\Users\aidan\Documents\PSPS' # Set this to a directory including the gps files (in their own subdirectories) and the csv
os.chdir(wd)
print(os.getcwd())

C:\Users\aidan\Documents\PSPS


In [20]:
df = pd.read_csv("data-2018-cleaned.csv") # Set this to the .csv file
print(df.shape)

(1168, 19)


In [21]:
# Cleaning for duplicates
df = df.drop_duplicates(['Date ', 'POI', 'PolygonLine ID'])
print(df.shape)
df.head(5)

(1059, 19)


,Date,Time in (24 hours),Time out (24 hours),Weather,Crew Leader (name),POI,General Location,Species Code,Point Area (m²) or Line Depth (m),PolygonLine ID,Direction from trail,Distribution Code,Does this cherry laurel or holly plant have a DBH of 20cm+?,Holly Berries present?,Proximity to stream or wetland,Vegetation layer,Native Herb Layer %,X_COORD,Y_COORD
0,2017-02-12,9:30:00,12:30:00,Rain,Wendy Thompson,2323,Sasamat,HB,2.53,NaN,W,3.0,NaN,NaN,10m+,Mixed,1-5%,NaN,NaN
1,2017-02-12,9:30:00,12:30:00,Rain,Wendy Thompson,2324,Sasamat,HB,5.00,L1-1,W,6.0,NaN,NaN,10m+,Mixed,1-5%,NaN,NaN
2,2017-02-12,9:30:00,12:30:00,Rain,Wendy Thompson,2325,Sasamat,HB,5.00,L2-2,W,6.0,NaN,NaN,10m+,Mixed,6-25%,NaN,NaN
3,2017-02-12,9:30:00,12:30:00,Rain,Wendy Thompson,2326,Sasamat,EI,0.50,NaN,W,1.0,NaN,NaN,10m+,Herb,6-25%,NaN,NaN
4,2017-02-12,9:30:00,12:30:00,Rain,Wendy Thompson,2327,Sasamat,EH,0.50,NaN,W,NaN,NaN,Unknown,10m+,Herb,1-5%,NaN,NaN


In [22]:
print(df['Date '].tolist())

['2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017-02-12', '2017

In [24]:
# Fixing the date formatting
def fix_dates(date):
    x = datetime.strptime(date, '%Y-%m-%d')
    x = str(datetime.date(x))
    return x

df['Date '].map(fix_dates)
print(df.head(5))
print(df['Date '].dtype)

        Date  Time in \n(24 hours) Time out\n(24 hours)  Weather   \
0  2017-02-12              9:30:00              12:30:00     Rain   
1  2017-02-12              9:30:00              12:30:00     Rain   
2  2017-02-12              9:30:00              12:30:00     Rain   
3  2017-02-12              9:30:00              12:30:00     Rain   
4  2017-02-12              9:30:00              12:30:00     Rain   

  Crew Leader (name)    POI General Location Species Code  \
0      Wendy Thompson  2323          Sasamat           HB   
1      Wendy Thompson  2324          Sasamat           HB   
2      Wendy Thompson  2325          Sasamat           HB   
3      Wendy Thompson  2326          Sasamat           EI   
4      Wendy Thompson  2327          Sasamat           EH   

   Point Area (m²) or\n Line Depth (m) PolygonLine ID Direction from trail   \
0                                 2.53            NaN                     W   
1                                 5.00           L1-1       

In [8]:
# Retrieving unique values
def get_unique_col(dataframe, col_name):
    # Consumes a dataframe and returns the unique values in the requested column
    # meant as part of UID creation for the data points
    unique = dataframe.drop_duplicates([col_name])
    unique = unique[col_name].tolist()
    return unique



In [9]:
# NOTE: PolygonLine ID is not the original name for the column in the example data!! The original name has unacceptable characters.
dates = get_unique_col(df, 'Date ')
for date in dates:
    x = df.loc[df['Date '] == date, ['POI']]
    x = x['POI'].tolist()
    for poi in x:
        y = df.loc[df['POI']== poi, ['PolygonLine ID']]
        y = y['PolygonLine ID'].tolist()
        if pd.isnull(y[0]):
            continue
        else:
            for i in y:
                print(y)
                    
                if (y[0])[0] == 'L':
                    print((y[0])[0])
                elif (y[0])[0] == 'P':
                    print((y[0])[0])
                else:
                    print((y[0])[0])

['L1-1']
L
['L2-2']
L
['L2-1']
L
['L2-2']
L
['L2-3']
L
['L3-1']
L
['L3-2']
L
['L4-1']
L
['L4-2']
L
['L5-1']
L
['L5-2']
L
['L6-1']
L
['L6-2']
L
['P1-1']
P
['P1-2']
P
['P1-3']
P
['P1-4']
P
['P1-5']
P
['P1-6']
P
['P1-7']
P
['P1-8']
P
['P1-9']
P
['L1-1']
L
['L1-2']
L
['L2-1']
L
['L2-2']
L
['L3-1']
L
['L3-2']
L
['L4-1']
L
['L4-2']
L
['L5-1']
L
['L5-2']
L
['L6-1']
L
['L6-1']
L
['L7-1']
L
['L7-2']
L
['L8-1']
L
['L8-2']
L
['L9-1']
L
['L9-1']
L
['L10-1']
L
['L10-1']
L
['L11-1']
L
['L11-2']
L
['L12-1']
L
['L12-2']
L
['L13-1']
L
['L13-2']
L
['L1-1']
L
['L1-2']
L
['L2-1']
L
['L2-2', nan]
L
['L2-2', nan]
L
['L3-1']
L
['L3-2']
L
['L2-2', nan]
L
['L2-2', nan]
L
['L4-1']
L
['L4-2']
L
['L1-1']
L
['L1-2']
L
['L1-1', nan]
L
['L1-1', nan]
L
['L1-2', nan, nan]
L
['L1-2', nan, nan]
L
['L1-2', nan, nan]
L
['L2-1', nan]
L
['L2-1', nan]
L
['L2-2', nan]
L
['L2-2', nan]
L
['L2-3', nan]
L
['L2-3', nan]
L
['L3-1', nan]
L
['L3-1', nan]
L
['L3-2', 'L2-2']
L
['L3-2', 'L2-2']
L
['L1-2', nan, nan]
L
['L1-2', nan, nan]


In [10]:
# Class structure of each device
os.listdir()
class Device:
    def __init__(self, gps_name, directory):
        self.name = gps_name
        self.direc = directory
    def get_files(self):
        walker = os.walk(self.direc)
        files = [x[2] for x in walker][0]
        return files

In [11]:
# Provides a list of existing folders to create Device class instances from
walker = os.walk(wd)

devices_list = [ (x[0])[len(wd)+1:]
                    for x in walker 
                    if len(x[0]) > len(wd)
               ]
print(devices_list)

['GPS1-2018', 'GPS2-2018']


In [44]:
# Creating a list of device instances from devices_list
devices = [Device(device, (wd + '\\' + device)) for device in devices_list]
print(devices)

[<__main__.Device object at 0x0000016C446AED68>, <__main__.Device object at 0x0000016C446AE9E8>]


In [51]:
# Reading the .gpx file data
import datetime

def deliver_gps_data(gps):
    
   
    os.chdir(gps.direc)
    cwd = gps.direc
    files = gps.get_files()
    column_names = ['Date ', 'POI', 'X_COORD', 'Y_COORD']
    gps_list = []

    for file in files:
        gpx_file = open(gps.direc + '\\' + file, 'r')
        gpx = gpxpy.parse(gpx_file)

        for waypoint in gpx.waypoints:

            new_row = {}
            new_row.update({column_names[0]: str(waypoint.time)[:10],
                            column_names[1]: int(waypoint.name),
                            column_names[2]: waypoint.longitude,
                            column_names[3]: waypoint.latitude})
            gps_list.append(new_row)
    gps_df = pd.DataFrame(gps_list)
    print(gps_df.head(5))
    dates = pd.unique(gps_df['Date '])

    gps_out = df.merge(gps_df, how='inner', on=['Date ', 'POI'])
    return gps_out

In [52]:
for device in devices:
    x = deliver_gps_data(device)
    print(x.head(5))


        Date    POI     X_COORD    Y_COORD
0  2017-12-02  2868 -123.209429  49.257523
1  2017-12-02  2869 -123.209333  49.257427
2  2017-12-02  2870 -123.209357  49.256911
3  2017-12-02  2871 -123.208671  49.256672
4  2017-12-02  2872 -123.209394  49.256458
        Date  Time in \n(24 hours) Time out\n(24 hours)   Weather   \
0  2017-10-14              9:30:00              12:30:00  Overcast   
1  2017-10-14              9:30:00              12:30:00  Overcast   
2  2017-10-14              9:30:00              12:30:00  Overcast   
3  2017-10-14              9:30:00              12:30:00  Overcast   
4  2017-10-14              9:30:00              12:30:00  Overcast   

  Crew Leader (name)    POI General Location Species Code  \
0        Doug Crocker  2604        Cleveland           EH   
1        Doug Crocker  2605        Cleveland           EH   
2        Doug Crocker  2606        Cleveland           EH   
3        Doug Crocker  2612        Cleveland           EH   
4        Doug Cr

In [55]:

final = pd.concat([(deliver_gps_data(devices[0])), (deliver_gps_data(devices[1]))], ignore_index=True)
final.head(5)

        Date    POI     X_COORD    Y_COORD
0  2017-12-02  2868 -123.209429  49.257523
1  2017-12-02  2869 -123.209333  49.257427
2  2017-12-02  2870 -123.209357  49.256911
3  2017-12-02  2871 -123.208671  49.256672
4  2017-12-02  2872 -123.209394  49.256458
        Date    POI     X_COORD    Y_COORD
0  2017-12-02  2323 -123.209534  49.258013
1  2017-12-02  2324 -123.209577  49.258062
2  2017-12-02  2325 -123.209533  49.257881
3  2017-12-02  2326 -123.209589  49.257871
4  2017-12-02  2327 -123.209599  49.257861


,Date,Time in (24 hours),Time out (24 hours),Weather,Crew Leader (name),POI,General Location,Species Code,Point Area (m²) or Line Depth (m),PolygonLine ID,...,Distribution Code,Does this cherry laurel or holly plant have a DBH of 20cm+?,Holly Berries present?,Proximity to stream or wetland,Vegetation layer,Native Herb Layer %,X_COORD_x,Y_COORD_x,X_COORD_y,Y_COORD_y
0,2017-10-14,9:30:00,12:30:00,Overcast,Doug Crocker,2604,Cleveland,EH,0.5,NaN,...,2.0,NaN,Unknown,10m+,Herb,1-5%,NaN,NaN,-123.221247,49.257856
1,2017-10-14,9:30:00,12:30:00,Overcast,Doug Crocker,2605,Cleveland,EH,2.0,NaN,...,6.0,NaN,Unknown,10m+,Herb,1-5%,NaN,NaN,-123.221109,49.257767
2,2017-10-14,9:30:00,12:30:00,Overcast,Doug Crocker,2606,Cleveland,EH,3.0,NaN,...,6.0,NaN,Unknown,10m+,Mixed,76-100%,NaN,NaN,-123.220887,49.257553
3,2017-10-14,9:30:00,12:30:00,Overcast,Doug Crocker,2612,Cleveland,EH,2.0,NaN,...,3.0,NaN,Unknown,10m+,Mixed,26-50%,NaN,NaN,-123.220771,49.257579
4,2017-10-14,9:30:00,12:30:00,Overcast,Doug Crocker,2613,Cleveland,EH,2.0,NaN,...,4.0,NaN,Unknown,10m+,Mixed,6-25%,NaN,NaN,-123.220846,49.257453


In [57]:
final.to_csv(wd + '\\' + 'out.csv')